In [1]:
#TD for to dos

# for timeline analysis : prophet

## needed details 
- visit_number: number of visits on the site or the number of times they bought sth on the site
- #city_id: cityname , zipcode or sth available ?#
- price & campaign_involved: most of all what does campaign_involved denote? sometimes it seems to give a discount sometimes not 
- article_id: the id seems to be wrong ( too long )

## importing libraries

In [2]:
# Importing the necessary libraries
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import glob
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict

## setting options

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)

In [4]:
#adjusting notebook width
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:90% !important; }</style>"))

In [5]:
#markdowntables layout

In [6]:
%%html
<style>
table {float:left}
table td, table th, table tr {text-align:left !important;}
</style>

# loading data

In [7]:
#sales dataframe
df_base = pd.read_csv("./Data/TTdata.csv")

//anaconda3/envs/nf2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
#copy for working
df = df_base.copy()

In [9]:
#dataframe with citynames
#https://developers.google.com/adwords/api/docs/appendix/geotargeting
df_geotargets = pd.read_csv("./Data/geotargets.csv") 

In [10]:
df_stock = pd.read_csv("./Data/daily_TTI.DE.csv")

## first look at the data

In [11]:
df.head()

,visit_id_n,unser_id_n,visit_number,date,article_name,article_id,ga_category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved
0,551960,553811,1.0,20180910.0,Geschenk Gutschein 10 EUR,200000000010,Herren,(not set),10.0,1.0,desktop,Firefox,Windows,Germany,Lower Saxony,1004478,Herren,Herren,Herren,Herren,False
1,242735,254033,1.0,20180104.0,Geschenkgutschein 10 EUR,200000000010,Damen/Geschenkgutscheine,(not set),10.0,1.0,desktop,Firefox,Windows,Germany,Lower Saxony,9048892,Herren,Herren,Herren,Herren,True
2,680200,620544,2.0,20180612.0,Geschenk Gutschein 10 EUR,200000000010,Damen/Geschenkgutscheine,(not set),10.0,1.0,desktop,Chrome,Windows,Germany,Berlin,1003854,Herren,Herren,Herren,Herren,True
3,44920,609593,46.0,20180913.0,Geschenk Gutschein 10 EUR,200000000010,Herren,(not set),10.0,1.0,mobile,Safari,iOS,Germany,Lower Saxony,1004564,Herren,Herren,Herren,Herren,True
4,704159,73553,2.0,20180729.0,Geschenk Gutschein 10 EUR,200000000010,Herren,(not set),10.0,1.0,desktop,Chrome,Windows,Germany,Bavaria,1004246,Herren,Herren,Herren,Herren,True


In [12]:
df.tail()

,visit_id_n,unser_id_n,visit_number,date,article_name,article_id,ga_category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved
3016738,33526,316730,2.0,20181122.0,Unifarbener Ösenschal,4260478143821,(not set),TT_Casual,36.97,1.0,mobile,Chrome,Android,Germany,Berlin,1003854,Home & Living,Vorhänge,Home & Living,Schuhe & Accessoires,True
3016739,520195,51192,16.0,20181201.0,Unifarbener Ösenschal,4260478143821,Home & Living/Vorhänge,TT_Casual,42.85,2.0,mobile,Safari,iOS,Germany,North Rhine-Westphalia,1004707,Home & Living,Vorhänge,Home & Living,Schuhe & Accessoires,True
3016740,18132,393215,2.0,20181124.0,Unifarbener Ösenschal,4260478143821,(not set),TT_Casual,37.39,2.0,mobile,Safari,iOS,Germany,Berlin,1003854,Home & Living,Vorhänge,Home & Living,Schuhe & Accessoires,True
3016741,7155,211171,NaN,NaN,NaN,anonymized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shirts unter 20 Euro,Shirts unter 20 Euro,Shirts unter 20 €,T-Shirts & Langarmshirts,NaN
3016742,7155,211171,NaN,NaN,NaN,cliqz.com/tracking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Damen,langarm,Damen,T-Shirts & Langarmshirts,NaN


In [13]:
df.shape

(3016743, 21)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3016743 entries, 0 to 3016742
Data columns (total 21 columns):
visit_id_n           int64
unser_id_n           int64
visit_number         float64
date                 float64
article_name         object
article_id           object
ga_category          object
brand                object
price                float64
quantity             float64
device_category      object
device_browser       object
device_os            object
country              object
region               object
city_id              object
product_line         object
product_group        object
product_line_r       object
product_group_r      object
campaign_involved    object
dtypes: float64(4), int64(2), object(15)
memory usage: 483.3+ MB


In [15]:
df.describe()

,visit_id_n,unser_id_n,visit_number,date,price,quantity
count,3.016743e+06,3.016743e+06,3.016732e+06,3.016732e+06,3.016732e+06,3.016732e+06
mean,4.102715e+05,3.108908e+05,1.058004e+01,2.018055e+07,2.668266e+01,1.018650e+00
std,2.366932e+05,1.795393e+05,2.962561e+01,4.429808e+03,1.790388e+01,1.586700e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,2.017110e+07,0.000000e+00,1.000000e+00
25%,2.053810e+05,1.551850e+05,1.000000e+00,2.018032e+07,1.445000e+01,1.000000e+00
50%,4.107240e+05,3.107390e+05,3.000000e+00,2.018071e+07,2.420000e+01,1.000000e+00
75%,6.152210e+05,4.659450e+05,9.000000e+00,2.018103e+07,3.361000e+01,1.000000e+00
max,8.201950e+05,6.226180e+05,1.616000e+03,2.019023e+07,2.512600e+02,1.000000e+01


In [16]:
df.sample(n=30)

,visit_id_n,unser_id_n,visit_number,date,article_name,article_id,ga_category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved
537240,213802,329744,17.0,20171210.0,Sweatjacke mit Sternen,4059491945808,Damen/Damenmode/Pullover & Sweatshirts/Sweatsh...,TT_Denim,18.91,1.0,mobile,Chrome,Android,Germany,Baden-Wurttemberg,1004054,Damen,Sweatshirts,Damen,Pullover & Sweatshirts,True
2054870,398343,552141,6.0,20181017.0,Langarmshirt mit Print,4060868325734,Langarmshirts,TT_Casual,12.49,1.0,tablet,Safari,iOS,Austria,Vienna,1000997,Damen,Langarmshirts,Damen,T-Shirts & Langarmshirts,True
2027498,592864,494241,39.0,20181012.0,Revolverheld: Hemd mit Stehkragen,4060868304982,Stehkragenhemd,TT_Denim,20.16,1.0,mobile,Safari,iOS,Germany,Bavaria,9048754,Baumwoll Hemden,Baumwoll Hemden,Herren,Hemden,True
2628052,425534,579522,8.0,20180826.0,Pullover mit Rollkanten,4062105359311,Herren/Herrenmode/Pullover & Sweatshirts/Pullover,TT_Denim,18.90,1.0,tablet,Chrome,Android,Germany,Brandenburg,(not set),Herren,Pullover,Herren,Pullover & Sweatshirts,False
302476,352351,587536,3.0,20171222.0,Strukturiertes Damen-Kleid,4059491307163,Damen/Damenmode/Kleider,TT_Denim,33.61,1.0,mobile,Chrome,Android,Germany,Bremen,1004332,Kurze Kleider,Kurze Kleider,Damen,Kleider,False
1958389,258820,341706,55.0,20190212.0,Parka mit Teddyfell,4060868229582,"Damen/Damenmode/Jacken, Mäntel & Westen/Mäntel...",TT_Denim,52.93,1.0,mobile,Chrome,Android,Germany,North Rhine-Westphalia,9048396,Damen,Mäntel & Parkas,Damen,"Jacken, Mäntel & Westen",True
2276280,266146,299236,3.0,20190209.0,Schlichtes Langarmshirt,4060868882336,Damen/Damenmode/T-Shirts & Langarmshirts/Langa...,TT_Casual,16.80,1.0,mobile,Chrome,Android,Germany,Baden-Wurttemberg,1004066,Damen,Langarmshirts,Damen,T-Shirts & Langarmshirts,True
2051915,142306,479718,55.0,20181006.0,Langarmshirt mit Kapuze,4060868323358,Herren/Herrenmode/Pullover & Sweatshirts/Sweat...,TT_Denim,15.12,1.0,tablet,Chrome,Android,Germany,Schleswig-Holstein,1004856,Herren,Sweatshirts,Herren,Pullover & Sweatshirts,True
525455,237849,271574,6.0,20171118.0,Hoodie mit Schrift-Applikation,4059491902054,Herren/Herrenmode/Pullover & Sweatshirts/Sweat...,TT_Denim,32.13,1.0,tablet,Safari,iOS,Germany,Baden-Wurttemberg,1003978,Herren,Sweatshirts,Herren,Pullover & Sweatshirts,True
2841115,583855,354274,13.0,20181003.0,Kariertes Hemd,4062105677354,Herren/Herrenmode/Hemden/Langarm,TT_Casual,15.12,1.0,desktop,Firefox,Windows,Germany,Hamburg,1004437,Karohemden,Karohemden,Herren,Hemden,True


### feature dictionary ( my impressions )

| Feature No. | Feature Name | Description |
| :-: | :-: | :-: |
| 1 | visit_id_n | Unique ID of a single shopping cart |
| 2 | unser_id_n | Unique ID of a user (misstyped obviously) |
| 3 | visit_number | Not entirely clear; either number of visits to the shop or number of total purchases |
| 4 | date | date the purchase was made |
| 5 | article_name | Name of the article that was bought |
| 6 | article_id | ID of the article that was bought |
| 7 | ga_category | Category the article falls in ( partly the path in the webshop to the article ) |
| 8 | brand | Brand of the article |
| 9 | price | Price of the article |
| 10 | quantity | Number of the article that were bought |
| 11 | device_category | Category the user device falls in |
| 12 | device_browser | Category the user browser falls in |
| 13 | device_os | Category the user OS falls in |
| 14 | country | country location of the user |
| 15 | region | region location of the user |
| 16 | city_id | geoID of the town location of the user |
| 17 | product_line | Product line of the article |
| 18 | product_group | Product group of the article |
| 19 | product_line_r | not clear what the r stands for; might stand for retail |
| 20 | product_group_r | not clear what the r stands for; might stand for retail |
| 21 | campaign_involved | not clear what the campaign stands for |

# data cleaning

In [17]:
#rename columns
df.rename(columns={'visit_id_n':'visit_id',
                          'unser_id_n':'user_id',
                          'ga_category':'category'}, inplace=True)

## visit id
After extensive research in a later part of the project it came up, that some visit id's were assigned to multiple users. Probably because the german and austrian webshop data were just put together and not filtered. As such define visit id as the sum of visit id and user id to get a unique number.

In [18]:
df["visit_id"] = df["visit_id"].apply(lambda x: str(x)) + df["user_id"].apply(lambda x: str(x))
df = df.astype({"visit_id": "int64"})

## getting rid of nans

In [19]:
df.isna().sum()

visit_id              0
user_id               0
visit_number         11
date                 11
article_name         11
article_id            0
category             11
brand                11
price                11
quantity             11
device_category      11
device_browser       11
device_os            11
country              11
region               11
city_id              11
product_line          0
product_group         0
product_line_r        0
product_group_r       0
campaign_involved    11
dtype: int64

In [20]:
df.dropna(inplace=True) #just 11 , so drop them

In [21]:
df.isna().sum()

visit_id             0
user_id              0
visit_number         0
date                 0
article_name         0
article_id           0
category             0
brand                0
price                0
quantity             0
device_category      0
device_browser       0
device_os            0
country              0
region               0
city_id              0
product_line         0
product_group        0
product_line_r       0
product_group_r      0
campaign_involved    0
dtype: int64

In [22]:
#number of (not set)
df.isin(['(not set)']).sum(axis=0)

visit_id                  0
user_id                   0
visit_number              0
date                      0
article_name              0
article_id                0
category              41935
brand                398925
price                     0
quantity                  0
device_category           0
device_browser            0
device_os                75
country                1178
region                 7731
city_id               93307
product_line          31227
product_group         31227
product_line_r        31227
product_group_r       31227
campaign_involved         0
dtype: int64

## filtering cities and putting them into the sales dataframe

In [23]:
df_geotargets.head()

,Criteria ID,Name,Canonical Name,Parent ID,Country Code,Target Type,Status
0,1000002,Kabul,"Kabul,Kabul,Afghanistan",9075393.0,AF,City,Active
1,1000003,Luanda,"Luanda,Luanda Province,Angola",9070431.0,AO,City,Active
2,1000004,The Valley,"The Valley,Anguilla",2660.0,AI,City,Active
3,1000010,Abu Dhabi,"Abu Dhabi,Abu Dhabi,United Arab Emirates",9041082.0,AE,City,Active
4,1000011,Ajman,"Ajman,Ajman,United Arab Emirates",9047096.0,AE,City,Active


In [24]:
#Criteria ID as index
df_geotargets_clean = df_geotargets.set_index("Criteria ID")

In [25]:
#take just the Name column
df_geotargets_clean = df_geotargets_clean["Name"]

In [26]:
#get a dictionary from the dataframe
geodic = df_geotargets_clean.to_dict()

In [27]:
#entry for not set ids
geodic[0] = "(not set)"

In [28]:
#replace not set with a number for a moment to change to int
df["city_id"] = df["city_id"].replace({"(not set)": 0})

In [29]:
#change city_id type to int
df = df.astype({'city_id': 'int64'})

In [30]:
#map cities to dataframe with new column city
df["city"] = df["city_id"].map(geodic)

In [31]:
#nans in cities by ids that were not found
df.isna().sum()

visit_id               0
user_id                0
visit_number           0
date                   0
article_name           0
article_id             0
category               0
brand                  0
price                  0
quantity               0
device_category        0
device_browser         0
device_os              0
country                0
region                 0
city_id                0
product_line           0
product_group          0
product_line_r         0
product_group_r        0
campaign_involved      0
city                 640
dtype: int64

In [32]:
#df[df["city"].isnull()]

In [33]:
#fill the nans with (not set)
df.fillna(value={"city": "(not set)"}, inplace=True)

In [34]:
#df[df["city"].isnull()]["country"].value_counts()

In [35]:
df_geotargets["Target Type"].value_counts()

Postal Code               48391
City                      38097
Neighborhood               4631
County                     3417
Municipality               2124
Province                   1131
District                    948
Region                      909
Congressional District      441
Airport                     395
Department                  240
State                       235
University                  219
Country                     213
City Region                 183
Governorate                 121
National Park                96
Borough                      88
Prefecture                   49
Okrug                        28
Canton                       26
Autonomous Community         22
TV Region                    14
Union Territory               7
Territory                     4
Name: Target Type, dtype: int64

In [36]:
# check of the postal codes for germany ( not useful )
df_geotargets[df_geotargets["Target Type"] == "Postal Code"][df_geotargets["Country Code"]=="DE"].head()

//anaconda3/envs/nf2/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Criteria ID,Name,Canonical Name,Parent ID,Country Code,Target Type,Status
66187,9041520,15306,"15306,Brandenburg,Germany",20227.0,DE,Postal Code,Active
66188,9041521,15526,"15526,Brandenburg,Germany",20227.0,DE,Postal Code,Active
66189,9041522,15230,"15230,Brandenburg,Germany",20227.0,DE,Postal Code,Active
66190,9041523,15234,"15234,Brandenburg,Germany",20227.0,DE,Postal Code,Active
66191,9041524,15236,"15236,Brandenburg,Germany",20227.0,DE,Postal Code,Active
66192,9041525,15890,"15890,Brandenburg,Germany",20227.0,DE,Postal Code,Active
66193,9041526,03172,"03172,Brandenburg,Germany",20227.0,DE,Postal Code,Active
66194,9041527,15848,"15848,Brandenburg,Germany",20227.0,DE,Postal Code,Active
66195,9041528,15907,"15907,Brandenburg,Germany",20227.0,DE,Postal Code,Active
66196,9041529,03205,"03205,Brandenburg,Germany",20227.0,DE,Postal Code,Active


In [37]:
#Brand seems inherintly useless 
df.brand.unique()

array(['(not set)', 'TT_Casual', 'TT_Denim'], dtype=object)

### price seems wrong for the following, as the original price is around 5-8, so discounts are in the price already

In [38]:
df.iloc[658646]

visit_id                                                  424670120783
user_id                                                         120783
visit_number                                                         4
date                                                       2.01807e+07
article_name                                  Bandana in Melange-Optik
article_id                                               4059953239858
category             Kinder/Kindermode/Baby Mädchen 0 - 17 Monate/A...
brand                                                        TT_Casual
price                                                             0.38
quantity                                                             1
device_category                                                desktop
device_browser                                                 Firefox
device_os                                                      Windows
country                                                        Germany
region

## change datatypes

In [39]:
#change visit_number and quantity to int as float does not make sense
df = df.astype({'visit_number': 'int64', "quantity": "int64", "article_id": "int64"})
#device stuff and brand to category
df = df.astype({'device_category': 'category', "device_browser": "category", "device_os": "category", "brand": "category"})
#campaign_involved to bool
df.campaign_involved = df.campaign_involved.astype("bool")

In [40]:
#change objects to string for starters
#df[df.select_dtypes('object').columns] = df[df.select_dtypes('object').columns].astype(np.str) 
#does not work

In [41]:
#date to date format
df["date"]= pd.to_datetime(df["date"],format='%Y%m%d')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3016732 entries, 0 to 3016740
Data columns (total 22 columns):
visit_id             int64
user_id              int64
visit_number         int64
date                 datetime64[ns]
article_name         object
article_id           int64
category             object
brand                category
price                float64
quantity             int64
device_category      category
device_browser       category
device_os            category
country              object
region               object
city_id              int64
product_line         object
product_group        object
product_line_r       object
product_group_r      object
campaign_involved    bool
city                 object
dtypes: bool(1), category(4), datetime64[ns](1), float64(1), int64(6), object(9)
memory usage: 428.7+ MB


## list of years to look at the span of time we have

In [43]:
year_list = pd.DatetimeIndex(df['date']).year  

In [44]:
year_list.unique()

Int64Index([2018, 2017, 2019], dtype='int64', name='date')

In [45]:
df.date.max()

Timestamp('2019-02-28 00:00:00')

In [46]:
df.date.min()

Timestamp('2017-11-01 00:00:00')

We have data from 2017-11-01 till 2019-02-28

## check for missing dates 

In [47]:
from datetime import date, timedelta
date_set = set(df.date.min() + timedelta(x) for x in range((df.date.max() - df.date.min()).days))
missing = sorted(date_set - set(df.date))
missing

[]

As we can see there are no missing days in the date Data.

## add days
monday 0 , ... , sunday 6

In [48]:
df["day_of_week"] = pd.to_datetime(df["date"]).dt.dayofweek

## article names and id

In [49]:
df.article_name.value_counts()

Alexa Slim Jeans                                  62458
Josh Regular Slim Jeans                           41092
T-Shirt mit Schrift-Print                         38895
T-Shirt mit Print                                 29257
T-Shirt mit Logo-Print                            28231
Marvin Straight Jeans                             27405
Alexa Straight Jeans                              22816
Damen-Basic-Pullover mit V-Ausschnitt             19512
Jim Slim Bermuda Shorts                           18002
Basic Herren-Pullover mit V-Ausschnitt            17812
Travis Regular Chino                              16859
Basic Herren-Strick-Pullover                      14865
gemustertes Blusen-Shirt                          14560
Gemusterte Bluse                                  14144
gemustertes T-Shirt                               13267
Jona Extra Skinny Jeans                           13179
Gestreifter Damen-Pullover                        13091
Damen-Straight-Jeans – Alexa                    

In [50]:
df.article_id.value_counts()

4059066698719    1728
4059066698511    1284
4002702235090    1215
4059066698726    1176
4059066698733    1054
4059953383155     998
4059953383148     908
4059953383162     887
4059953383179     861
4059491897060     819
4060586142743     799
4060586158591     791
4058114237092     789
4002702235083     783
4059953383049     781
4052691223851     781
4054985602212     765
4052691223844     751
4002702235076     748
4060586158607     743
4060586142781     725
4060586230112     718
4059066698672     709
4059491897077     697
4062105707020     682
4059953383186     678
4059491843289     677
4059491843302     675
4062105681436     668
4056762815792     659
4059491784308     658
4059491520005     653
4058114237108     651
4062105692074     648
4054985761858     646
4059491784315     645
4060586158614     636
4054985761865     632
4059491843296     631
4057655756987     628
4060586270033     627
4059491776532     625
4060586262861     621
4059953863794     617
4059953383193     613
4056762815

## change the names of the following to match

In [51]:
d = {'Geschenk Gutschein 10 EUR':'Geschenkgutschein 10 EUR','Geschenk Gutschein 20 EUR':'Geschenkgutschein 20 EUR','Geschenk Gutschein 50 EUR':'Geschenkgutschein 50 EUR'}
df = df.replace(d)

## duplicated rows

In [52]:
df.duplicated().sum()

632

In [53]:
duplicateRowsDF = df[df.duplicated(keep=False)]

In [54]:
duplicateRowsDF.head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved,city,day_of_week
118,22892413921,413921,45,2018-07-11,Geschenkgutschein 20 EUR,200000000020,Herren,(not set),20.00,1,desktop,Chrome,Windows,Austria,Upper Austria,1000827,Herren,Herren,Herren,Herren,False,Linz,2
119,22892413921,413921,45,2018-07-11,Geschenkgutschein 20 EUR,200000000020,Herren,(not set),20.00,1,desktop,Chrome,Windows,Austria,Upper Austria,1000827,Herren,Herren,Herren,Herren,False,Linz,2
675,177945413228,413228,1,2019-01-01,Sneaker-Socken im Doppelpack,4002702123007,"Kinder/Kindermode/Mädchen 1,5 - 7 Jahre/Schuh...",TT_Casual,2.31,1,mobile,Chrome,Android,Germany,Berlin,1003854,Kinder,Schuhe & Accessoires,Kinder,Schuhe & Accessoires,True,Berlin,1
676,177945413228,413228,1,2019-01-01,Sneaker-Socken im Doppelpack,4002702123007,"Kinder/Kindermode/Mädchen 1,5 - 7 Jahre/Schuh...",TT_Casual,2.31,1,mobile,Chrome,Android,Germany,Berlin,1003854,Kinder,Schuhe & Accessoires,Kinder,Schuhe & Accessoires,True,Berlin,1
1010,601457557264,557264,1,2018-12-25,Sneaker-Socken im Doppelpack,4002702123014,"Kinder/Kindermode/Mädchen 1,5 - 7 Jahre/Schuh...",TT_Casual,3.18,10,mobile,Chrome,iOS,Germany,Baden-Wurttemberg,1004022,Kinder,Schuhe & Accessoires,Kinder,Schuhe & Accessoires,False,Rastatt,1


Note that it is not sure whether these rows are truly duplicates or just multiple purchases where the quantity wasn't set properly or the articles have the same name and price and for example different colors which this data does not tell us.

In [55]:
#df.drop_duplicates(keep="first", inplace=True)
#df.reset_index()

checking number of visits to number of entries (45 for this user)

In [56]:
df.user_id[413921]

230677

In [57]:
df[df['user_id'] == 413921].shape

(27, 23)

## duplicates in visit_id

In [58]:
df.visit_id.max()

820195135969

In [59]:
df.visit_id.duplicated().sum()

2181687

In [60]:
df_visits = df[df.duplicated(subset="visit_id") == True]

In [61]:
df_uniquevisits = df.drop_duplicates(subset="visit_id")

In [62]:
df_visits.shape

(2181687, 23)

In [63]:
df_uniquevisits.shape

(835045, 23)

In [64]:
df_visits.sort_values(by=['visit_id']).head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved,city,day_of_week
2972494,234440,40,1,2018-11-18,Schlichter Snood,4062105772851,Hoodies/Herren,TT_Denim,17.99,1,mobile,Chrome,Android,Austria,Lower Austria,1000750,Herren,Sweatshirts,Herren,Pullover & Sweatshirts,True,Melk,6
2817928,234440,40,1,2018-11-18,Hoodie mit Print,4062105670652,Hoodies/Herren,TT_Casual,15.00,1,mobile,Chrome,Android,Austria,Lower Austria,1000750,Hoodies,Herren,Herren,Herren,True,Melk,6
2538377,234440,40,1,2018-11-18,Basic Herren-Strick-Pullover,4062105188904,Herren,TT_Casual,14.99,1,mobile,Chrome,Android,Austria,Lower Austria,1000750,Herren,Pullover,Herren,Pullover & Sweatshirts,True,Melk,6
2375327,276103,76103,22,2019-02-21,Culver Skinny Jeans,4061945274082,Herren/Herrenmode/Jeans/Skinny,TT_Denim,42.01,1,mobile,Safari,iOS,Germany,Berlin,1003854,Herren,Skinny,Herren,Jeans,True,Berlin,3
2647054,290200,200,43,2018-11-03,Steppjacke,4062105383903,Steppjacken,TT_Casual,31.51,1,mobile,Chrome,Android,Germany,Brandenburg,1003866,Herbst Outfits,Herbst Outfits,Herbst Outfits,Herbst Outfits,True,Frankfurt an der Oder,5


In [65]:
df_uniquevisits.sort_values(by=["visit_id"]).head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved,city,day_of_week
320852,234440,40,1,2018-11-18,Basic Herren-Strick-Pullover,4059491520210,Herren,TT_Casual,14.99,1,mobile,Chrome,Android,Austria,Lower Austria,1000750,Herren,Pullover,Herren,Pullover & Sweatshirts,True,Melk,6
2079241,276103,76103,22,2019-02-21,Henley T-Shirt,4060868345305,Herren/Herrenmode/T-Shirts & Langarmshirts/T-S...,TT_Casual,10.91,1,mobile,Safari,iOS,Germany,Berlin,1003854,Herren,T-Shirts,Herren,T-Shirts & Langarmshirts,True,Berlin,3
2645748,290200,200,43,2018-11-03,Steppjacke,4062105383569,Steppjacken/Damen,TT_Casual,31.50,1,mobile,Chrome,Android,Germany,Brandenburg,1003866,Damen,Jacken,Damen,"Jacken, Mäntel & Westen",True,Frankfurt an der Oder,5
2830805,336120,20,2,2018-10-25,Nela Extra Skinny Jeans,4062105674247,Damen/Damenmode/Jeans/Skinny,TT_Denim,33.61,1,mobile,Chrome,Android,Germany,Saarland,9048444,Damen,Skinny,Damen,Jeans,True,Homburg,3
321178,366022,6022,8,2017-12-30,Basic Herren-Strick-Pullover,4059491520241,Feeling Blue Men,TT_Casual,16.80,1,desktop,Safari,Macintosh,Germany,Berlin,1003854,Feeling Blue Men,Feeling Blue Men,Feeling Blue Men,Feeling Blue Men,True,Berlin,5


Duplicates in the visit_id surely denote multiple articles that were bought at the same visit, thus dropping any will not make sense. Also there were 820194 unique visit_id's, hence we know that the data consists of this number of shopping cart purchases.

## unique devices 
lets check the entries and number of occurance of the elements of the device features

In [68]:
df.device_browser.value_counts()

Chrome                      994731
Safari                      793300
Firefox                     589700
Samsung Internet            256273
Edge                        155931
Internet Explorer            88370
Android Webview              63918
Opera                        29834
Safari (in-app)              28517
Amazon Silk                  14662
YaBrowser                      354
SeaMonkey                      337
Puffin                         235
Maxthon                        217
Android Browser                 94
Mozilla Compatible Agent        85
Iron                            58
UC Browser                      43
Playstation 4                   26
BlackBerry                      14
MRCHROME                        12
Seznam                          12
Coc Coc                          4
Opera 9.4                        3
Nintendo Browser                 2
Name: device_browser, dtype: int64

In [69]:
df.device_category.value_counts()

desktop    1371610
mobile     1091610
tablet      553512
Name: device_category, dtype: int64

In [70]:
df.device_os.value_counts()

Windows          1204420
Android           923303
iOS               718614
Macintosh         156569
Linux              11633
Chrome OS           1778
BlackBerry           222
(not set)             75
Windows Phone         70
Playstation 4         26
Tizen                 19
Nintendo WiiU          2
FreeBSD                1
Name: device_os, dtype: int64

Nothing too surprising so far. Later we can check if there were differences in the purchases between the different features.

## campaign_involved:

In [71]:
camp_true = df[df.campaign_involved == True]

In [72]:
len(camp_true)

2278279

In [73]:
camp_true.sample(n=20).head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved,city,day_of_week
2145111,67073857439,57439,1,2019-01-08,Schlichtes Blusenshirt,4060868494959,Damen/Damenmode/Blusen/Langarm,TT_Casual,22.05,1,desktop,Chrome,Windows,Germany,North Rhine-Westphalia,1004636,Damen,Damen,Damen,Damen,True,Grevenbroich,1
203547,156451590761,590761,24,2018-03-25,Sneaker mit Metallic-Details,4058219388705,Damen/Damenschuhe & Accessoires/Schuhe,(not set),32.32,1,desktop,Firefox,Windows,Germany,Hesse,1004422,Damen,Schuhe,Damen,Schuhe & Accessoires,True,Russelsheim,6
2437131,466646298926,298926,15,2018-07-18,Gemusterte Kurzarmbluse,4062105065571,Damen/Damenmode/Blusen/kurzarm,TT_Casual,16.80,1,tablet,Chrome,Android,Germany,Baden-Wurttemberg,1004066,Damen,kurzarm,Damen,T-Shirts & Langarmshirts,True,Villingen-Schwenningen,2
686617,332814312259,312259,69,2018-01-19,Pullover mit V-Ausschnitt,4059953296301,Damen/Damenmode/Pullover & Sweatshirts/Pullover,TT_Casual,26.88,1,desktop,Chrome,Linux,Germany,Lower Saxony,1004536,Damen,Neu im Sale,Damen,Neu im Sale,True,Oldenburg,4
2158059,707616520581,520581,5,2018-10-08,Karierte Bluse,4060868502746,Damen/Damenmode/Blusen/Langarm,TT_Casual,20.16,1,tablet,Safari,iOS,Germany,Bavaria,1004314,Damen,Langarm,Damen,T-Shirts & Langarmshirts,True,Vilshofen an der Donau,0


In [74]:
camp_false = df[df.campaign_involved == False]

In [75]:
len(camp_false)

738453

In [76]:
camp_false[camp_false["article_id"]==4011694579266].head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved,city,day_of_week
35271,505889355464,355464,5,2017-11-13,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,15.12,1,mobile,Firefox,Android,Germany,Berlin,1003854,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,False,Berlin,0
35274,383063313091,313091,2,2018-03-24,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,18.90,1,tablet,Safari,iOS,Germany,Schleswig-Holstein,9048708,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,False,Sankt Peter-Ording,5
35277,201946434776,434776,6,2018-07-29,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,25.20,1,desktop,Firefox,Windows,Germany,Schleswig-Holstein,1004866,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,False,Kiel,6
35281,333769606504,606504,1,2018-09-17,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,20.16,1,desktop,Chrome,Macintosh,Germany,Berlin,1003854,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,False,Berlin,0
35295,14858564131,564131,1,2019-02-04,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,16.80,1,desktop,Firefox,Macintosh,Germany,Lower Saxony,1004536,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,False,Oldenburg,0


In [77]:
camp_true[camp_true["article_id"]==4011694579266].head()

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved,city,day_of_week
35270,723099348206,348206,6,2017-11-13,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,21.42,1,mobile,Samsung Internet,Android,Germany,Berlin,1003854,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,True,Berlin,0
35272,539377527591,527591,6,2018-08-29,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,25.21,1,mobile,Samsung Internet,Android,Germany,Bavaria,1004269,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,True,Rosenheim,2
35273,54340111696,111696,2,2018-09-08,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,18.90,1,desktop,Chrome,Chrome OS,Germany,Lower Saxony,9048874,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,True,Wiesmoor,5
35275,492320350403,350403,6,2018-10-26,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,14.70,2,desktop,Chrome,Windows,Germany,North Rhine-Westphalia,1004761,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,True,Wermelskirchen,4
35276,31699418198,418198,1,2018-09-13,Hip-Pants im 3er-Pack,4011694579266,Herren/Herrenmode/Underwear & Pyjamas,TT_Casual,18.90,1,mobile,Chrome,Android,Germany,Thuringia,1004999,Herren,Underwear & Pyjamas,Herren,Underwear & Pyjamas,True,Suhl,3


we see, that campaign_involved can not mean discounts

## nr. of occurence of countries

In [78]:
df.country.value_counts()

Germany                  2756836
Austria                   196950
Netherlands                22547
Switzerland                 5425
Belgium                     2434
Czechia                     2293
Poland                      2174
United Kingdom              2021
Italy                       1956
United States               1850
Spain                       1827
Hungary                     1766
Slovenia                    1457
France                      1425
Slovakia                    1366
Romania                     1331
(not set)                   1178
Estonia                     1038
Denmark                      969
Croatia                      900
Luxembourg                   852
Ukraine                      842
Lithuania                    804
Latvia                       710
Sweden                       514
Greece                       507
Turkey                       466
Bulgaria                     348
United Arab Emirates         258
Portugal                     248
Ireland   

## columns for article type

In [79]:
#put all features describing the article type together for easier use
df["genre_column_placeholder"] = df["article_name"] + "/" + df["category"] + "/" + df["product_line"] + "/" + df["product_line_r"] + "/" + df["product_group"] + "/" + df["product_group_r"]

In [80]:
df.genre_column_placeholder = df.genre_column_placeholder.astype("str")

In [81]:
#new column for articles of women genre
def check_women(item):
    return "" if re.search(r"([Dd]{1}ame[ns]{1})|([Ww]{1}om[ea]{1}n)", item) == None else "Women"

df['women_article'] = df['genre_column_placeholder'].apply(check_women)


In [82]:
#new column for articles of men genre
def check_men(item):
    return "" if re.search(r"([Hh]{1}e[r]{1,2}en)|(Men\b)|(\b[Mm]en\b)", item) == None else "Men"

df['men_article'] = df['genre_column_placeholder'].apply(check_men)


In [83]:
#new column for articles of kids/teenager/babies genre
def check_kids(item):
    return "" if re.search(r"([Kk]{1}inder)|(\b[Kk]{1}id[s]{0,1})|([Jj]{1}ungen)|([Mm]{1}ädchen)|([Tt]{1}een[s]{0,1})|([Bb]{1}aby)|([Bb]{1}abies)", item) == None else "Kids"

df['kids_article'] = df['genre_column_placeholder'].apply(check_kids)


In [84]:
#new column for articles of home&living genre
def check_hl(item):
    return "" if re.search(r"([Hh]{1}ome)|([Ll]iving)", item) == None else "Home&living"

df['homeandliving_article'] = df['genre_column_placeholder'].apply(check_hl)


In [85]:
df = df.assign(category_slim = df.women_article + df.men_article + df.kids_article + df.homeandliving_article)

In [86]:
def putslash(item): 
  
    # regex [A-Z][a-z]* means any string starting  
    # with capital character followed by many  
    # lowercase letters  
    words = re.findall('[A-Z][a-z]*', item) 
  
    # Change first letter of each word into lower 
    # case 
    result = [] 
    for word in words: 
        word = chr( ord (word[0]) + 32) + word[1:] 
        result.append(word) 
    return '/'.join(result)

In [87]:
df["category_slim"] = df["category_slim"].apply(putslash)

In [88]:
df["category_slim"] = df["category_slim"].replace("", "(not set)")

In [90]:
df.women_article = df.women_article.replace({"": 0, "Women": 1})
df.men_article = df.men_article.replace({"": 0, "Men": 1})
df.kids_article = df.kids_article.replace({"": 0, "Kids": 1})
df.homeandliving_article = df.homeandliving_article.replace({"": 0, "Home&living": 1})

In [120]:
article_diff = df[(df.women_article + df.men_article + df.kids_article + df.homeandliving_article) == 0]

In [121]:
article_diff.product_group_r.value_counts()

Schuhe & Accessoires        7878
(not set)                   7284
Herbst Outfits              6384
T-Shirts & Langarmshirts    1042
Neu im Sale                  427
Festival Styles              326
SAFARI STYLES                212
Summer Sale Highlights       121
Bold Summer Colors            83
Black & White                 46
Alles zum halben Preis        24
T-Shirts unter 20 €            9
Partyoutfits                   3
Revolverheld Kollektion        2
Mexico Styles                  2
Name: product_group_r, dtype: int64

In [122]:
article_diff.category.value_counts()

(not set)                                  7000
Herbst Outfits                             4431
Athleisure                                 3881
Schuhe                                     1805
Winterpullover                              818
Business-Kleider                            462
Neu im Sale                                 420
Baumwollkleidung                            403
Gürtel                                      380
Shirts unter 20 Euro                        377
Baumwollhosen                               364
Taschen/medium                              324
Kurzblazer                                  259
Slim                                        232
SAFARI STYLES                               190
Sonnenbrillen                               185
medium                                      152
Festivalstyles                              146
Taschen                                     138
T-Shirts unter 20 Euro                       91
Summer Sale Highlights                  

In [123]:
len(article_diff)

23843

So we are left with only 23843 articles without any of the new article groups.

In [94]:
#number of articles that got multiple article groups
mult_article = df[df.women_article + df.men_article + df.kids_article + df.homeandliving_article > 1]

In [95]:
mult_article.sample(n=20)

,visit_id,user_id,visit_number,date,article_name,article_id,category,brand,price,quantity,device_category,device_browser,device_os,country,region,city_id,product_line,product_group,product_line_r,product_group_r,campaign_involved,city,day_of_week,genre_column_placeholder,women_article,men_article,kids_article,homeandliving_article,category_slim
1361734,311721312008,312008,4,2018-03-15,Lässige Damen-Stoffhose,4060586270026,Damen/Damenmode/Hosen,TT_Denim,26.88,1,desktop,Firefox,Macintosh,Germany,Bavaria,1004246,sommerhosen,sommerhosen,Herren,Hosen,False,Nuremberg,3,Lässige Damen-Stoffhose/Damen/Damenmode/Hosen/...,1,1,0,0,women/men
257887,67382209390,209390,1,2019-01-09,Travis Regular Chino,4059066344227,Herren/Herrenmode/Hosen,TT_Casual,28.35,1,desktop,Chrome,Macintosh,Germany,Berlin,1003854,chino hosen,chino hosen,Damen,Hosen,False,Berlin,2,Travis Regular Chino/Herren/Herrenmode/Hosen/c...,1,1,0,0,women/men
2113977,634027560197,560197,1,2018-12-28,Nela Extra Skinny Jeans,4060868464655,Damen/Damenmode/Jeans/Skinny,TT_Denim,45.16,1,tablet,Safari,iOS,Germany,Schleswig-Holstein,1004872,Baumwollhosen,Baumwollhosen,Herren,Hosen,False,Neumunster,4,Nela Extra Skinny Jeans/Damen/Damenmode/Jeans/...,1,1,0,0,women/men
2681721,33135984437,84437,3,2018-12-23,Alexa Slim Hose,4062105491066,Damen/Damenmode/Hosen,TT_Casual,33.61,1,tablet,Safari,iOS,Germany,Hamburg,1004437,Baumwollhosen,Baumwollhosen,Herren,Hosen,True,Hamburg,6,Alexa Slim Hose/Damen/Damenmode/Hosen/Baumwoll...,1,1,0,0,women/men
1972972,270592569585,569585,3,2018-09-02,Steppjacke,4060868238713,"Damen/Damenmode/Jacken, Mäntel & Westen/Jacken",TT_Denim,58.82,1,desktop,Edge,Windows,Germany,Bavaria,1004150,Steppjacken,Damen,Herren,Damen,True,Erding,6,"Steppjacke/Damen/Damenmode/Jacken, Mäntel & We...",1,1,0,0,women/men
2768834,96238216622,216622,1,2018-10-10,Winter Parka,4062105566351,Winterparkas/Herren,TT_Casual,67.22,1,tablet,Safari,iOS,Germany,North Rhine-Westphalia,1004764,Winterparkas,Winterparkas,Damen,"Jacken, Mäntel & Westen",True,Wiehl,2,Winter Parka/Winterparkas/Herren/Winterparkas/...,1,1,0,0,women/men
1225858,401271291386,291386,1,2018-12-24,gemustertes Kurzarmhemd,4060586194452,Herren/Herrenmode/Hemden/Kurzarm,TT_Casual,15.96,1,mobile,Chrome,iOS,Germany,Bavaria,1004142,Sommerkleidung,Sommerkleidung,Damen,Kleider,False,Dorfen,0,gemustertes Kurzarmhemd/Herren/Herrenmode/Hemd...,1,1,0,0,women/men
2018498,229197164115,164115,7,2018-10-27,Kleid mit Taillenbund,4060868300090,Business-Mode/Damen,TT_Casual,28.00,1,tablet,Safari,iOS,Germany,Lower Saxony,1004507,Business-Mode,Business-Mode,Herren,Business,True,Hanover,5,Kleid mit Taillenbund/Business-Mode/Damen/Busi...,1,1,0,0,women/men
2677410,760650154549,154549,4,2018-09-22,Kunstlederjacke,4062105464695,Uebergangsjacken/Damen,TT_Casual,83.32,1,mobile,Chrome,Android,Austria,Vienna,1000997,Uebergangsjacken,Damen,Herren,Damen,True,Vienna,5,Kunstlederjacke/Uebergangsjacken/Damen/Ueberga...,1,1,0,0,women/men
2677065,164887272909,272909,4,2018-09-11,Kunstlederjacke,4062105464602,Sommerjacken,TT_Casual,67.22,1,mobile,Chrome,Android,Germany,Hesse,1004363,Uebergangsjacken,Damen,Herren,Damen,True,Frankfurt,1,Kunstlederjacke/Sommerjacken/Uebergangsjacken/...,1,1,0,0,women/men


In [96]:
len(mult_article)

65808

Having 65808 articles with multiple article groups. Most of them do make at least some sense, like a jacket for a teenager also having the product_line Men.

## total price per article

In [97]:
df["total_price"] = df["price"] * df["quantity"]

## ordering columns in a proper way

In [99]:
#code for checking the index of a certain column
df.columns.get_loc("category_slim")

28

In [100]:
#check name of certain index
df.columns[20]

'campaign_involved'

In [101]:
#reordering the columns
cols = df.columns.tolist()
cols = cols[0:4] + [cols[22]] + cols[4:6] + cols[8:10] + [cols[29]] \
        + cols[13:15] + [cols[21]] + [cols[15]] + cols[24:28] + [cols[20]] \
        + [cols[6]] + [cols[28]] + cols[7:8] + cols[16:20] + [cols[23]] + cols[10:13]
print("Check if any column was skipped when reordering (True: all columns fetched):", len(cols) == len(df.columns))
df = df.reindex(columns = cols)

Check if any column was skipped when reordering (True: all columns fetched): True


## adding value and number of items in the shopping cart of a visit

In [102]:
shopcart = df.groupby(["visit_id"]).sum()

In [103]:
#take just the Name column
df_shopcartvalue_clean = shopcart["total_price"]
df_shopcartnumber_clean = shopcart["quantity"]

In [104]:
#get a dictionary from the dataframe
cartdicval = df_shopcartvalue_clean.to_dict()
cartdicnum = df_shopcartnumber_clean.to_dict()
cartdiffart = df.visit_id.value_counts().to_dict()

In [105]:
df["shop_cart_value"] = df["visit_id"].map(cartdicval)
df["shop_cart_item_count"] = df["visit_id"].map(cartdicnum)
df["shop_cart_unique_articles"] = df["visit_id"].map(cartdiffart)

In [107]:
#reordering the columns
cols = df.columns.tolist()
cols = cols[0:10] + cols[30:33] + cols[10:30]
print("Check if any column was skipped when reordering (True: all columns fetched):", len(cols) == len(df.columns))
df = df.reindex(columns = cols)

Check if any column was skipped when reordering (True: all columns fetched): True


## add stockprice per day to the dataframe

In [109]:
df_stock["timestamp"]= pd.to_datetime(df_stock["timestamp"],format='%Y-%m-%d')

In [110]:
#fill missing days ( weekends )
df_stock = df_stock.resample('D',on="timestamp").mean()

In [111]:
#fill the nans of the missings above with the average of their direct set neighbours
df_stock = (df_stock.ffill() + df_stock.bfill())/2

In [112]:
df_stock.head()

,open,high,low,close,volume
timestamp,,,,,
2010-03-26,11.9637,12.04200,11.4576,11.8257,2767746.0
2010-03-27,11.8740,11.97755,11.5404,11.8349,1489778.5
2010-03-28,11.8740,11.97755,11.5404,11.8349,1489778.5
2010-03-29,11.7843,11.91310,11.6232,11.8441,211811.0
2010-03-30,11.8073,11.91770,11.6600,11.8717,80096.0


In [113]:
#take just the close column
df_stock_clean = df_stock["close"]

In [114]:
#get a dictionary from the dataframe
stockdic = df_stock_clean.to_dict()

In [115]:
#map stock to dataframe with new column stock_value
df["stock_value"] = df["date"].map(stockdic)

In [116]:
#check for nans that were not set 
df.isna().sum()

visit_id                     0
user_id                      0
visit_number                 0
date                         0
day_of_week                  0
article_name                 0
article_id                   0
price                        0
quantity                     0
total_price                  0
shop_cart_value              0
shop_cart_item_count         0
shop_cart_unique_articles    0
country                      0
region                       0
city                         0
city_id                      0
women_article                0
men_article                  0
kids_article                 0
homeandliving_article        0
campaign_involved            0
category                     0
category_slim                0
brand                        0
product_line                 0
product_group                0
product_line_r               0
product_group_r              0
genre_column_placeholder     0
device_category              0
device_browser               0
device_o

In [117]:
#easy dataframe to check the stockprice development 
df.drop_duplicates("date").sort_values("date").head(20)

,visit_id,user_id,visit_number,date,day_of_week,article_name,article_id,price,quantity,total_price,shop_cart_value,shop_cart_item_count,shop_cart_unique_articles,country,region,city,city_id,women_article,men_article,kids_article,homeandliving_article,campaign_involved,category,category_slim,brand,product_line,product_group,product_line_r,product_group_r,genre_column_placeholder,device_category,device_browser,device_os,stock_value
2325,14930374362,374362,3,2017-11-01,2,Tennis-Socken im Dreierpack,4002702234727,5.01,1,5.01,32.69,5,5,Germany,North Rhine-Westphalia,Neuss,1004708,0,1,0,0,True,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Strümpfe & Socken,Herren,Schuhe & Accessoires,Tennis-Socken im Dreierpack/Herren/Herrenschuh...,mobile,Chrome,Android,8.8590
5836,645245525600,525600,2,2017-11-02,3,Basic Socken im Doppelpack,4002702235090,5.84,2,11.68,108.55,5,4,Germany,Baden-Wurttemberg,(not set),0,0,1,0,0,True,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Strümpfe & Socken,Herren,Schuhe & Accessoires,Basic Socken im Doppelpack/Herren/Herrenschuhe...,desktop,Firefox,Windows,9.1310
2849,448437326618,326618,1,2017-11-03,4,Tennis-Socken im Dreierpack,4002702234765,5.52,2,11.04,83.18,7,6,Germany,North Rhine-Westphalia,Wermelskirchen,1004761,0,1,0,0,False,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Strümpfe & Socken,Herren,Schuhe & Accessoires,Tennis-Socken im Dreierpack/Herren/Herrenschuh...,desktop,Firefox,Windows,9.1000
2512,813967575820,575820,11,2017-11-04,5,Tennis-Socken im Dreierpack,4002702234758,6.68,2,13.36,78.38,4,3,Germany,Baden-Wurttemberg,Esslingen,1003938,0,1,0,0,True,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Strümpfe & Socken,Herren,Schuhe & Accessoires,Tennis-Socken im Dreierpack/Herren/Herrenschuh...,mobile,Samsung Internet,Android,9.0900
455,754081156504,156504,1,2017-11-05,6,Geschenkgutschein 50 EUR,200000000050,50.00,1,50.00,50.00,1,1,Germany,Schleswig-Holstein,Flensburg,1004855,0,1,0,0,False,Herren,men,(not set),Herren,Herren,Herren,Herren,Geschenkgutschein 50 EUR/Herren/Herren/Herren/...,desktop,Firefox,Windows,9.0900
357,817688334365,334365,4,2017-11-06,0,Geschenkgutschein 50 EUR,200000000050,50.00,1,50.00,50.00,1,1,Germany,Mecklenburg-Vorpommern,Gustrow,1004447,0,1,0,0,True,Herren,men,(not set),Herren,Herren,Herren,Herren,Geschenkgutschein 50 EUR/Herren/Herren/Herren/...,desktop,Chrome,Macintosh,9.0800
4025,403206555145,555145,3,2017-11-07,1,Basic Socken im Doppelpack,4002702235076,5.84,2,11.68,115.76,9,5,Germany,North Rhine-Westphalia,Munster,1004707,0,1,0,0,True,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Strümpfe & Socken,Herren,Schuhe & Accessoires,Basic Socken im Doppelpack/Herren/Herrenschuhe...,desktop,Firefox,Linux,9.4490
2574,81446267456,267456,2,2017-11-08,2,Tennis-Socken im Dreierpack,4002702234758,4.49,2,8.98,63.59,7,4,Germany,Bavaria,Strullendorf,9048780,0,1,0,0,True,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Strümpfe & Socken,Herren,Schuhe & Accessoires,Tennis-Socken im Dreierpack/Herren/Herrenschuh...,desktop,Firefox,Windows,9.4410
2526,777162451382,451382,1,2017-11-09,3,Tennis-Socken im Dreierpack,4002702234758,6.68,1,6.68,20.12,2,2,Germany,Bavaria,Nuremberg,1004246,0,1,0,0,False,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Strümpfe & Socken,Herren,Schuhe & Accessoires,Tennis-Socken im Dreierpack/Herren/Herrenschuh...,desktop,Internet Explorer,Windows,8.9500
2314,321355221002,221002,1,2017-11-10,4,Tennis-Socken im Dreierpack,4002702234727,6.68,1,6.68,38.56,3,3,Germany,Hesse,Frankfurt,1004363,0,1,0,0,False,Herren/Herrenschuhe & Accessoires/Accessoires/...,men,TT_Casual,Herren,Strümpfe & Socken,Herren,Schuhe & Accessoires,Tennis-Socken im Dreierpack/Herren/Herrenschuh...,desktop,Firefox,Windows,9.2770


## features for first date of purchase and a feature for first or established purchase

In [118]:
#help feature with a edited date
df['year_month'] = df['date'].map(lambda date: 100*date.year + date.month)

In [119]:
#finding the date where users bought for the first time in our timeline
min_purchase = df.groupby("user_id").year_month.min().reset_index()
min_purchase.columns = ["user_id","min_purchase_date"]

#merge
df = pd.merge(df, min_purchase, on="user_id")

In [125]:
#create a feature user_type with first or established depending on time of purchase
df["user_type"] = "first"
df.loc[df["year_month"]>df["min_purchase_date"], "user_type"] = "established"

### feature dictionary after some work

| Feature No. | Feature Name | Description |
| :-: | :-: | :-: |
| 1 | visit_id | Unique ID consisting of the ID of a single shopping cart and the ID of the user |
| 2 | user_id | Unique ID of a user |
| 3 | visit_number | Not entirely clear; either number of visits to the shop or number of total purchases |
| 4 | date | Date the purchase was made |
| 5 | day_of_week | Day when the purchase was made; 0 for monday till 6 for sunday |
| 6 | article_name | Name of the article that was bought |
| 7 | article_id | ID of the article that was bought |
| 8 | price | Price of the article |
| 9 | quantity | Number of the article that were bought |
| 10 | total_price | Total price of the article purchase consisting of price * quantity |
| 11 | shop_cart_value | Value of the whole shopping cart of a single visit_id |
| 12 | shop_cart_item_count | Number of items in the shopping cart of a single visit_id |
| 13 | shop_cart_unique_articles | Number of unique items in the shopping cart of a single visit_id |
| 14 | country | country location of the user |
| 15 | region | region location of the user |
| 16 | city | city location of the user |
| 17 | city_id | geoID of the town location of the user |
| 18 | women_article | women category - 1 yes , 0 no |
| 19 | men_article | men category - 1 yes , 0 no |
| 20 | kids_article | kids category - 1 yes , 0 no |
| 21 | homeandliving_article | home & living category - 1 yes , 0 no |
| 22 | campaign_involved | still not completely clear but after some research it might stand for the cases where users came to the shop via advertisement. When you get to the shop this way the url contains a part for campaign: for example "utm_campaign=de_retargeting_criteo"|
| 23 | category | Category the article falls in ( partly the path in the webshop to the article ) |
| 24 | category_slim | featured category with women, men, kids and home&living |
| 25 | brand | Brand of the article |
| 26 | product_line | Product line of the article |
| 27 | product_group | Product group of the article |
| 28 | product_line_r | not clear what the r stands for; might stand for retail |
| 29 | product_group_r | not clear what the r stands for; might stand for retail |
| 30 | genre_column_placeholder | Sum of article name, category and product lines/groups to better work with |
| 31 | device_category | Category the user device falls in |
| 32 | device_browser | Category the user browser falls in |
| 33 | device_os | Category the user OS falls in |
| 34 | stock_value | company stock price at this day |
| 35 | year_month | feature with year and month as int |
| 36 | min_purchase_date | date of the first purchase in our timeline of this user |
| 37 | user_type | type of the purchase, first or established |

## save dataframe to pickle

In [130]:
df.to_pickle('Data/dataframefull.pickle')